In [1]:
# This uses PynPoint on Altair data as a test

# created 2018 Sept. 23 by E.S.

In [1]:
import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.IOmodules.TextReading import ParangReadingModule
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ImageResizing, \
                                       ContrastCurveModule

In [2]:
working_place = "./pynpoint_experimentation_altair/working_place/"
input_place = "./pynpoint_experimentation_altair/input_place/"
output_place = "./pynpoint_experimentation_altair/output_place/"

#url = urllib.URLopener()
#url.retrieve("https://people.phys.ethz.ch/~stolkert/BetaPic_NACO_Mp.hdf5",
#             input_place+"BetaPic_NACO_Mp.hdf5")

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

Initiating PynPoint... [DONE]


In [3]:
# read in science FITS files

read_science = FitsReadingModule(name_in="read_science",
                                 input_dir=None,
                                 image_tag="science",
                                 check=True)

pipeline.add_module(read_science)

In [4]:
# read in PSF reference FITS files (i.e., unsaturated frames)

read_ref_psf = FitsReadingModule(name_in="read_ref_psf",
                                 input_dir=input_place+'ref_psf/',
                                 image_tag="ref_psf",
                                 check=True)

pipeline.add_module(read_ref_psf)

In [5]:
# prepare PSF for PCA-based subtraction

prep = PSFpreparationModule(name_in="prep",
                            image_in_tag="science",
                            image_out_tag="prep",
                            image_mask_out_tag=None,
                            mask_out_tag=None,
                            norm=False,
                            resize=2,
                            cent_size=0.15,
                            edge_size=1.1)

pipeline.add_module(prep)

In [6]:
# do PCA-based PSF subtraction

pca = PcaPsfSubtractionModule(pca_numbers=(20, ),
                              name_in="pca",
                              images_in_tag="prep",
                              reference_in_tag="prep",
                              res_mean_tag="residuals")

pipeline.add_module(pca)

In [7]:
# make a contrast curve

detec = ContrastCurveModule(name_in="contrast",
                            image_in_tag="science",
                            psf_in_tag="ref_psf",
                            contrast_out_tag="contrast_landscape")

pipeline.add_module(detec)

In [ ]:
pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute EXP_NO (=ESO DET EXP NO) not found in the FITS header.
  "header." % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute ND

Running FitsReadingModule... [DONE]
Running FitsReadingModule... [DONE]
Running PSFpreparationModule... [DONE]
Constructing PSF model... [DONE]
Creating residuals... [DONE]
Running ContrastCurveModule...
Processing position 1 out of 540.

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/ProcessingModules/DetectionLimits.py:164: UserWarning: The number of frames in psf_in_tag does not match with the number of frames in image_in_tag. Using the mean of psf_in_tag as PSF template.
  warnings.warn('The number of frames in psf_in_tag does not match with the number of '


In [10]:
test = pipeline.get_attribute("science", "PARANG")

KeyError: "Can't open attribute (Can't locate attribute: 'parang')"

In [14]:
test = pipeline.get_attribute("science", "CPU")

KeyError: "Can't open attribute (Can't locate attribute: 'cpu')"

In [12]:
test

0.027

In [7]:
residuals = pipeline.get_data("residuals")
pixscale = pipeline.get_attribute("science", "PIXSCALE")
contrastdata = pipeline.get_attribute("contrast_landscape")

size = pixscale*np.size(residuals, 1)/2.

In [8]:
plt.imshow(residuals[0, ], origin='lower', extent=[size, -size, -size, size])
plt.title("Altair test")
plt.xlabel('R.A. offset [arcsec]', fontsize=12)
plt.ylabel('Dec. offset [arcsec]', fontsize=12)
plt.colorbar()
#plt.show()
plt.savefig(output_place+"residuals.png")